<a href="https://colab.research.google.com/github/stirmizi7/Final-Project/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bloxs


In [ ]:
pip install plotly


In [ ]:
pip install category-encoders

In [ ]:
import re
import math
import random
import numpy as np
import pandas as pd
import time

import gc

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

from bloxs import B
import plotly.express as px
import missingno

from category_encoders import TargetEncoder

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

<ipython-input-4-18b5c9b1e179>:17: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
dtypes = {

    # numeric columns containing corrupted values
    'Age':'object',
    'Num_of_Loan':'object',
    'Num_of_Delayed_Payment':'object',
    'Annual_Income':'object',
    'Outstanding_Debt':'object',
    'Amount_invested_monthly':'object',
    'Monthly_Balance':'object',

    # numeric columns with descrete values
    'Num_Bank_Accounts':pd.Int16Dtype(),
    'Num_Credit_Card':pd.Int16Dtype(),
    'Interest_Rate':pd.Int16Dtype(),

    # numeric columns with continues values
    'Delay_from_due_date':pd.Float32Dtype(),
    'Monthly_Inhand_Salary':pd.Float32Dtype(),
    'Changed_Credit_Limit':pd.Float32Dtype(),
    'Num_Credit_Inquiries':pd.Float32Dtype(),
    'Credit_Utilization_Ratio':pd.Float32Dtype(),
    'Total_EMI_per_month':pd.Float32Dtype(),

    # categorical columns
    'ID':'category',
    'Customer_ID':'category',
    'Month':'category',
    'Name':'category',
    'Occupation':'category',
    'Payment_of_Min_Amount':'category',

    # columns that need further processing
    'Payment_Behaviour':'string',
    'Credit_History_Age':'string',
    'SSN':'string',
    'Type_of_Loan':'string'
}

missing_values = ['_______', '_', '!@9#%8', '#F%$D@*&8']



In [ ]:
df_train=pd.read_csv('Resources/train.csv',  na_values = missing_values, dtype = dtypes)
df_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,11.27,4.0,NaN,809.98,26.82262,22 Years and 1 Months,No,49.574947,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1.0,NaN,11.27,4.0,Good,809.98,31.94496,<NA>,No,49.574947,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,<NA>,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574947,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5.0,4,6.27,4.0,Good,809.98,31.377861,22 Years and 4 Months,No,49.574947,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6.0,NaN,11.27,4.0,Good,809.98,24.797346,22 Years and 5 Months,No,49.574947,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [ ]:
col_names=df_train.columns
col_names

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [ ]:
columns_to_drop = ["Name", "Age", "Month", "Occupation",'ID', 'SSN','Customer_ID']
df_train.drop(columns=columns_to_drop, inplace=True)
df_train.head()

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,11.27,4.0,NaN,809.98,26.82262,22 Years and 1 Months,No,49.574947,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1.0,NaN,11.27,4.0,Good,809.98,31.94496,<NA>,No,49.574947,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,<NA>,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574947,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5.0,4,6.27,4.0,Good,809.98,31.377861,22 Years and 4 Months,No,49.574947,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6.0,NaN,11.27,4.0,Good,809.98,24.797346,22 Years and 5 Months,No,49.574947,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [ ]:
def odd_value(col):
    df_train[col]=df_train[col].str.strip('_')
    df_train[col]=df_train[col].replace(['!@9#%8','', 'nan'],np.NaN)
cols=list(df_train.select_dtypes(include='O').columns)
list(map(odd_value,cols))

[None, None, None, None, None, None, None, None]

In [ ]:
def change_dtype(col,dtype):
    df_train[col]=df_train[col].astype(dtype)

required_features=['Annual_Income','Num_of_Loan','Delay_from_due_date','Num_of_Delayed_Payment','Changed_Credit_Limit','Outstanding_Debt','Amount_invested_monthly','Monthly_Balance']
for col in required_features:
    change_dtype(col,'float64')

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Annual_Income             100000 non-null  float64 
 1   Monthly_Inhand_Salary     84998 non-null   Float32 
 2   Num_Bank_Accounts         100000 non-null  Int16   
 3   Num_Credit_Card           100000 non-null  Int16   
 4   Interest_Rate             100000 non-null  Int16   
 5   Num_of_Loan               100000 non-null  float64 
 6   Type_of_Loan              88592 non-null   string  
 7   Delay_from_due_date       100000 non-null  float64 
 8   Num_of_Delayed_Payment    92998 non-null   float64 
 9   Changed_Credit_Limit      97909 non-null   float64 
 10  Num_Credit_Inquiries      98035 non-null   Float32 
 11  Credit_Mix                79805 non-null   object  
 12  Outstanding_Debt          100000 non-null  float64 
 13  Credit_Utilization_Ratio  1000

In [ ]:
print(df_train.columns)

Index(['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')


In [ ]:
encoding_variables=['Credit_Mix','Payment_of_Min_Amount','Payment_Behaviour']
def frequency_encoding(df_train, column):
    freq_map = df_train[column].value_counts(normalize=True).to_dict()
    df_train[column + '_freq_enc'] = df_train[column].map(freq_map)
for column in encoding_variables:
    frequency_encoding(df_train, column)
df_train['Credit_Mix']=df_train['Credit_Mix_freq_enc']
df_train['Payment_of_Min_Amount']=df_train['Payment_of_Min_Amount_freq_enc']
df_train['Payment_Behaviour']=df_train['Payment_Behaviour_freq_enc']
df_train.drop(columns=['Credit_Mix_freq_enc','Payment_of_Min_Amount_freq_enc','Payment_Behaviour_freq_enc'],inplace=True)
df_train.head()

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,19114.12,1824.843384,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7.0,11.27,4.0,NaN,809.98,26.82262,22 Years and 1 Months,0.35667,49.574947,80.415295,0.122727,312.494089,Good
1,19114.12,<NA>,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1.0,NaN,11.27,4.0,0.304956,809.98,31.94496,<NA>,0.35667,49.574947,118.280222,0.112825,284.629162,Good
2,19114.12,<NA>,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7.0,NaN,4.0,0.304956,809.98,28.609352,22 Years and 3 Months,0.35667,49.574947,81.699521,0.150011,331.209863,Good
3,19114.12,<NA>,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",5.0,4.0,6.27,4.0,0.304956,809.98,31.377861,22 Years and 4 Months,0.35667,49.574947,199.458074,0.276115,223.451310,Good
4,19114.12,1824.843384,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",6.0,NaN,11.27,4.0,0.304956,809.98,24.797346,22 Years and 5 Months,0.35667,49.574947,41.420153,0.189827,341.489231,Good


In [ ]:
if 'Type_of_Loan' in df_train.columns:
    df_train.drop(columns=['Type_of_Loan'], inplace=True)
else:
    print("Column 'Type_of_Loan' not found.")

In [ ]:
df_train.dropna(inplace=True)
df_train.drop(columns=['Credit_History_Age'], inplace=True)

In [ ]:
X = df_train.drop(columns=['Credit_Score'])
y = df_train['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7669024339504888


In [ ]:
df_test=pd.read_csv('Resources/test.csv',  na_values = missing_values, dtype = dtypes)
df_test.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,11.27,2022.0,Good,809.98,35.030403,22 Years and 9 Months,No,49.574947,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,9,13.27,4.0,Good,809.98,33.053116,22 Years and 10 Months,No,49.574947,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1.0,4,12.27,4.0,Good,809.98,33.811893,<NA>,No,49.574947,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",4.0,5,11.27,4.0,Good,809.98,32.430557,23 Years and 0 Months,No,49.574947,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,NaN,34847.84,3037.986572,2,4,6,1,Credit-Builder Loan,3.0,1,5.42,5.0,Good,605.03,25.926821,27 Years and 3 Months,No,18.816216,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [ ]:
columns_to_drop = ["Name", "Age", "Month", "Occupation",'ID', 'SSN','Customer_ID']
df_test.drop(columns=columns_to_drop, inplace=True)
df_test.head()

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7,11.27,2022.0,Good,809.98,35.030403,22 Years and 9 Months,No,49.574947,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,9,13.27,4.0,Good,809.98,33.053116,22 Years and 10 Months,No,49.574947,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,19114.12,1824.843384,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1.0,4,12.27,4.0,Good,809.98,33.811893,<NA>,No,49.574947,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,19114.12,<NA>,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",4.0,5,11.27,4.0,Good,809.98,32.430557,23 Years and 0 Months,No,49.574947,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,34847.84,3037.986572,2,4,6,1,Credit-Builder Loan,3.0,1,5.42,5.0,Good,605.03,25.926821,27 Years and 3 Months,No,18.816216,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [ ]:
def odd_value(col):
    df_test[col]=df_test[col].str.strip('_')
    df_test[col]=df_test[col].replace(['!@9#%8','', 'nan'],np.NaN)
cols=list(df_test.select_dtypes(include='O').columns)
list(map(odd_value,cols))

[None, None, None, None, None, None, None]

In [ ]:
def change_dtype(col,dtype):
    df_test[col]=df_train[col].astype(dtype)

required_features=['Annual_Income','Num_of_Loan','Delay_from_due_date','Num_of_Delayed_Payment','Changed_Credit_Limit','Outstanding_Debt','Amount_invested_monthly','Monthly_Balance']
for col in required_features:
    change_dtype(col,'float64')

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Annual_Income             23990 non-null  float64 
 1   Monthly_Inhand_Salary     42502 non-null  Float32 
 2   Num_Bank_Accounts         50000 non-null  Int16   
 3   Num_Credit_Card           50000 non-null  Int16   
 4   Interest_Rate             50000 non-null  Int16   
 5   Num_of_Loan               23990 non-null  float64 
 6   Type_of_Loan              44296 non-null  string  
 7   Delay_from_due_date       23990 non-null  float64 
 8   Num_of_Delayed_Payment    23990 non-null  float64 
 9   Changed_Credit_Limit      23990 non-null  float64 
 10  Num_Credit_Inquiries      48965 non-null  Float32 
 11  Credit_Mix                40195 non-null  object  
 12  Outstanding_Debt          23990 non-null  float64 
 13  Credit_Utilization_Ratio  50000 non-null  Floa

In [ ]:
df_test.dropna(inplace=True)
df_test.drop(columns=['Credit_History_Age'], inplace=True)
df_test.head()

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
9,34847.84,12187.219727,1,5,8,1.0,"Auto Loan, Auto Loan, and Not Specified",7.0,1.0,7.42,3.0,Good,605.03,35.685837,No,246.992325,40.391238,Low_spent_Large_value_payments,484.591214
12,34847.84,2612.490723,2,5,4,1.0,Not Specified,3.0,1.0,6.42,4.0,Good,605.03,39.37722,No,16.415451,130.115420,High_spent_Medium_value_payments,444.867032
15,34847.84,2612.490723,2,5,4,1.0,Not Specified,3.0,4.0,5.42,7.0,Good,605.03,35.579266,No,16.415451,218.904344,Low_spent_Small_value_payments,356.078109
25,30689.89,10469.208008,0,1,8,1.0,"Credit-Builder Loan, and Mortgage Loan",5.0,3.0,1.99,4.0,Good,632.46,31.025572,No,911.220154,124.881820,Low_spent_Large_value_payments,409.951812
26,30689.89,10469.208008,0,1,8,1.0,"Credit-Builder Loan, and Mortgage Loan",3.0,9.0,1.99,4.0,Good,632.46,26.066641,NM,911.220154,83.406509,Low_spent_Medium_value_payments,411.427123


In [ ]:
encoding_variables=['Credit_Mix','Payment_of_Min_Amount','Payment_Behaviour']
def frequency_encoding(df_test, column):
    freq_map = df_test[column].value_counts(normalize=True).to_dict()
    df_test[column + '_freq_enc'] = df_test[column].map(freq_map)
for column in encoding_variables:
    frequency_encoding(df_test, column)
df_test['Credit_Mix']=df_test['Credit_Mix_freq_enc']
df_test['Payment_of_Min_Amount']=df_test['Payment_of_Min_Amount_freq_enc']
df_test['Payment_Behaviour']=df_test['Payment_Behaviour_freq_enc']
df_test.drop(columns=['Credit_Mix_freq_enc','Payment_of_Min_Amount_freq_enc','Payment_Behaviour_freq_enc'],inplace=True)
df_test.head()

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
9,34847.84,12187.219727,1,5,8,1.0,"Auto Loan, Auto Loan, and Not Specified",7.0,1.0,7.42,3.0,0.268299,605.03,35.685837,0.317178,246.992325,40.391238,0.115436,484.591214
12,34847.84,2612.490723,2,5,4,1.0,Not Specified,3.0,1.0,6.42,4.0,0.268299,605.03,39.37722,0.317178,16.415451,130.115420,0.189378,444.867032
15,34847.84,2612.490723,2,5,4,1.0,Not Specified,3.0,4.0,5.42,7.0,0.268299,605.03,35.579266,0.317178,16.415451,218.904344,0.281826,356.078109
25,30689.89,10469.208008,0,1,8,1.0,"Credit-Builder Loan, and Mortgage Loan",5.0,3.0,1.99,4.0,0.268299,632.46,31.025572,0.317178,911.220154,124.881820,0.115436,409.951812
26,30689.89,10469.208008,0,1,8,1.0,"Credit-Builder Loan, and Mortgage Loan",3.0,9.0,1.99,4.0,0.268299,632.46,26.066641,0.119336,911.220154,83.406509,0.148548,411.427123


In [ ]:
print(df_test.columns)

Index(['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Payment_of_Min_Amount',
       'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour',
       'Monthly_Balance'],
      dtype='object')


In [ ]:
if 'Type_of_Loan' in df_test.columns:
    df_test.drop(columns=['Type_of_Loan'], inplace=True)

In [ ]:
df_test_scaled= scaler.transform(df_test)
predicted_scores = model.predict(df_test_scaled)
df_test['Credit_score'] = predicted_scores
df_test.head()


,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_score
9,34847.84,12187.219727,1,5,8,1.0,7.0,1.0,7.42,3.0,0.268299,605.03,35.685837,0.317178,246.992325,40.391238,0.115436,484.591214,Standard
12,34847.84,2612.490723,2,5,4,1.0,3.0,1.0,6.42,4.0,0.268299,605.03,39.37722,0.317178,16.415451,130.115420,0.189378,444.867032,Good
15,34847.84,2612.490723,2,5,4,1.0,3.0,4.0,5.42,7.0,0.268299,605.03,35.579266,0.317178,16.415451,218.904344,0.281826,356.078109,Standard
25,30689.89,10469.208008,0,1,8,1.0,5.0,3.0,1.99,4.0,0.268299,632.46,31.025572,0.317178,911.220154,124.881820,0.115436,409.951812,Good
26,30689.89,10469.208008,0,1,8,1.0,3.0,9.0,1.99,4.0,0.268299,632.46,26.066641,0.119336,911.220154,83.406509,0.148548,411.427123,Good


In [ ]:
pip install keras-tuner

In [ ]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                                    min_value=1,
                                                    max_value=10,
                                                    step=2), activation=activation, input_dim=X_train_scaled.shape[1]))
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                        min_value=1,
                                                        max_value=10,
                                                        step=2),
                                          activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    return nn_model

In [ ]:
import keras_tuner as kt
import tensorflow as tf

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
print("y_train data type:", type(y_train))
print("y_test data type:", type(y_test))
print("y_train shape:", y_train.shape)

y_train data type: <class 'pandas.core.series.Series'>
y_test data type: <class 'pandas.core.series.Series'>
y_train shape: (38454,)


In [ ]:
print("Unique values in y_train:", np.unique(y_train))

Unique values in y_train: ['Good' 'Poor' 'Standard']


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
print("Unique values in y_train after encoding:", np.unique(y_train))

Unique values in y_train after encoding: [0 1 2]


In [ ]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))